Importovanie knižníc, funkcie na úpravu dátumu a nastavenie adresy pre ElasticSearch

In [3]:
from bs4 import BeautifulSoup
import re
import os
import json
import calendar
import glob
from elasticsearch import Elasticsearch
import datetime
abbr_to_num = {name: num for num, name in enumerate(calendar.month_abbr) if num}
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [4]:
with open('Analyzers/analyzer.txt', 'r') as myfile:
    analyzer=myfile.read()
es.indices.create(index='goodreads-test', ignore=400, body=analyzer)

with open('Analyzers/french.txt', 'r') as myfile:
    analyzer_f=myfile.read()
es.indices.create(index='goodreads-test-f', ignore=400, body=analyzer_f)
    
with open('Analyzers/german.txt', 'r') as myfile:
    analyzer_g=myfile.read()
es.indices.create(index='goodreads-test-g', ignore=400, body=analyzer_g)
    
with open('Analyzers/spanish.txt', 'r') as myfile:
    analyzer_s=myfile.read()
es.indices.create(index='goodreads-test-s', ignore=400, body=analyzer_s)


with open('mapper/mapping.txt', 'r') as myfile:
    mapping=myfile.read()
with open('mapper/mapping_f.txt', 'r') as myfile:
    mapping_f=myfile.read()
with open('mapper/mapping_g.txt', 'r') as myfile:
    mapping_g=myfile.read()
with open('mapper/mapping_s.txt', 'r') as myfile:
    mapping_s=myfile.read()

    
es.indices.put_mapping(index='goodreads-test',body=mapping)
es.indices.put_mapping(index='goodreads-test-f',body=mapping_f)
es.indices.put_mapping(index='goodreads-test-s',body=mapping_s)
es.indices.put_mapping(index='goodreads-test-g',body=mapping_g)

{'acknowledged': True}

Funkcia na extrahovanie informácií z HTML stánky spolu s ošetrením, ak sa daný údaj na stránke nenachádza.

In [5]:
def parse_langs(html):
#     print(html)
#     try

    try:
        lang = re.search('(itemprop=\"inLanguage\">)(([a-z]|[A-Z]).*)\<',html).group(2)
        print(lang)
    except: 
        lang = "Undef"
    x = {
      "lang": lang}
    return json.dumps(x)

In [6]:
def parse_html(html):
#     print(html)
#     try

    try:
        lang = re.search('(itemprop=\"inLanguage\">)(([a-z]|[A-Z]).*)\<',html).group(2)
#         print(lang)
    except: 
        lang = "Undef"
    try:
        ident = re.search('\/book\/show\/([0-9]*)',html).group(1)
#         print(ident)
    except: 
        ident = None
    try:
        description = re.search('id=\"description\"(.*\n){3}.*\"\>(.*)\<\/',html).group(1)
#         print(description)
    except: 
        try:
            description = re.search('id=\"description\"(.*\n){1}.*\"\>(.*)\<\/',html).group(2)
        except:
            description = None
    try:
#         description = description.replace('<[^>]*>', ' ')
        description = re.sub('<[^>]*>', " ", description)
    except: 
        description = None
    try:
        name = re.search('\"bookTitle.*\n\s*(.*)',html).group(1)
#         name = re.search('(?<=(Original Title<\/div>\n)).*(?=\ \<)',html)
#         print(name)
    except: 
        name = None
#         print("sas")
    try:
        isbn = re.search('(?<=(ISBN<\/div>\n)).*\n\s*(.*)',html).group(2)
    except: 
        isbn = None
    try:
        isbn13 = re.search('(?<=(ISBN<\/div>\n)).*\n.*\n.*((?<=\>).*)(?=\<span)[^0-9]*([0-9]+)',html).group(3)
    except: 
        isbn13 = None
    try:
        series = re.search('(?<=(Series<\/div>\n))((\s*\<d|\s*\<a).*\n){0,10}\s*<\/div>',html).group()
    except: 
        series = None
    try:
        series = re.findall('\>([a-zA-Z0-9#\ -]*)\<\/a',series)
    except: 
        series = None
    try:
        characters = re.findall('\/characters.{1,40}"\>([a-zA-z0-9\ ]*)\<',html)
    except: 
        characters = None
#     obj= make_BookObj(ident, name, isbn, isbn13, series, characters)
#     Object.assign(obj, characters)

#     return obj
#     print(create_json(ident, name, isbn, isbn13, series, description, characters))
    return create_json(ident, lang , name, isbn, isbn13, series, description, characters)

In [7]:
class BookObj(object):
    ident: 0
    name: None
    isbn: None
    isbn13: None
    series: None
    lang: None
#          description: description
    characters: None

    def __init__(self, lang, ident, name, isbn, isbn13, series, characters):
        self.ident: ident
        self.name: name
        self.isbn: isbn
        self.isbn13: isbn13
        self.series: series
        self.lang: lang
#          self.description: description
        self.characters: characters

def make_BookObj(ident, name, isbn, isbn13, series, characters):
    book = BookObj(ident, name, isbn, isbn13, series, characters)
    return book

Funkcia zo vstupných údajov vytvorí JSON formát, a vlozi ho do elastic search databazy.

In [8]:
def create_json(ident, lang, name, isbn, isbn13, series, description, characters):
    x = {
      "id": ident,
      "lang": lang,
      "name": name,
      "isbn": isbn,
      "isbn13": isbn13,
      "series": series,
      "description": description,
      "characters": characters
    }
    y = json.dumps(x)
    global es
    page_id=0
    page_id += 1
    return y
#     print(json.dumps(json.loads(y), indent=2))

Spracovanie všetkých HMTL súborov uložených v adresári.

In [9]:
folders = os.listdir("D:/School/VINF/Data")
# folders = os.listdir("D:/School/VINF/DATA-TEST")
arr = []
arr_g = []
arr_s = []
arr_f = []
count = 0
# langs= []
for folder in folders:
    count=count+1
    if count% 250 == 0:
        print(count)
    book = {}
    book_g = {}
    book_s = {}
    book_f = {}
    arr_g.clear()
    arr_s.clear()
    arr_f.clear()
    arr.clear()
    book['book'] = folder
    book_g['book'] = folder
    book_s['book'] = folder
    book_f['book'] = folder
    files = glob.glob("D:/School/VINF/Data/"+folder+"/*.html")
#     files = glob.glob("D:/School/VINF/DATA-TEST/"+folder+"/*.html")
    for file in files:
        html = open(file,encoding="utf8")    
        soup = html.read()
#         print(soup)
        obj = (json.loads(parse_html(soup)))
#         lnagsObj = (json.loads(parse_langs(soup)))
        identity= re.search('([0-9]*)\.',file).group(1)
        obj['id'] = identity

        arr.append(obj)
        if obj['lang'] == "German":
            arr_g.append(obj)
        elif obj['lang'] == "Spanish":
            arr_s.append(obj)
        elif obj['lang'] == "French":
            arr_f.append(obj)
        
#         langs.append(lnagsObj)

        if arr:
            book['editions']=arr
        if arr_g:
            book_g['editions']=arr_g
        if arr_s:
            book_s['editions']=arr_s
        if arr_f:
            book_f['editions']=arr_f
#         print(arr)

    try:
        book['editions']
        jsonObj=json.dumps(book, indent=4, )
        es.index(index='goodreads-test', id=identity, body=jsonObj)
#         print('EEE')
    except:
        book={}
    try:
        book_f['editions']
        jsonObj_f=json.dumps(book_f, indent=4, )
        es.index(index='goodreads-test-f', id=identity, body=jsonObj_f)
#         print('FFF')
    except:
        book_f={}
    try:
        book_s['editions']
        jsonObj_s=json.dumps(book_s, indent=4, )
        es.index(index='goodreads-test-s', id=identity, body=jsonObj_s)
#         print('SSS')
    except:
        book_s={}
    try:
        book_g['editions']
#         print('GGG')
        jsonObj_g=json.dumps(book_g, indent=4, )
        es.index(index='goodreads-test-g', id=identity, body=jsonObj_g)
    except:
        book_g={}
        
#     es.index(index='goodreads-test', id=ident, body=jsonObj)
#     print(jsonObj)
#     print("  ")
#     break

250
500
750
1000
1250
1500
1750
2000
2250
2500
2750
3000
3250
3500
3750
4000
4250
4500
4750
5000
5250
5500
5750
6000
6250
6500
6750
7000
7250
7500
7750
8000
8250
8500
8750
9000
9250
9500
9750
10000
10250
10500
10750
11000
11250
11500
11750
12000
12250
12500
12750
13000
13250
13500


In [37]:
import numpy as np

In [45]:
np.unique(np.array(list(map(lambda x: x['lang'], langs))), return_counts=True)

(array(['Arabic', 'Chinese', 'Danish', 'Dutch', 'English',
        'English, Middle (1100-1500)', 'French', 'German',
        'Greek, Ancient (to 1453)', 'Greek, Modern (1453-)', 'Italian',
        'Japanese', 'Ladino', 'Latin', 'Multiple languages',
        'Munda languages', 'Norwegian', 'Portuguese', 'Russian', 'Serbian',
        'Spanish', 'Turkish', 'Undef'], dtype='<U27'),
 array([    3,    12,     2,     3, 14080,     2,   148,   182,    10,
            2,    12,    44,     1,     4,    21,     1,     1,    11,
            8,     1,   394,     1,  1956], dtype=int64))